Start by importing some standard libraries for processing, as well as folium. We also verify that we have the right version.

In [111]:
import os
import numpy as np
import pandas as pd
import json
import folium
from branca.colormap import linear
folium.__version__ == '0.5.0'
pd.options.mode.chained_assignment = None  # default='warn'

## Task 1:

Go to the [eurostat](http://ec.europa.eu/eurostat/data/database) website and try to find a dataset that includes the european unemployment rates at a recent date.

Use this data to build a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) which shows the unemployment rate in Europe at a country level. Think about [the colors you use](https://carto.com/academy/courses/intermediate-design/choose-colors-1/), how you decided to [split the intervals](http://gisgeography.com/choropleth-maps-data-classification/) into data classes or which interactions you could add in order to make the visualization intuitive and expressive. Compare Switzerland's unemployment rate to that of the rest of Europe.


## Task 2:

Go to the [amstat](https://www.amstat.ch) website to find a dataset that includes the unemployment rates in Switzerland at a recent date.

> *HINT* Go to the `details` tab to find the raw data you need. If you do not speak French, German or Italian, think of using free translation services to navigate your way through. 

Use this data to build another Choropleth map, this time showing the unemployment rate at the level of swiss cantons. Again, try to make the map as expressive as possible, and comment on the trends you observe.

The Swiss Confederation defines the rates you have just plotted as the number of people looking for a job divided by the size of the active population (scaled by 100). This is surely a valid choice, but as we discussed one could argue for a different categorization.

Copy the map you have just created, but this time don't count in your statistics people who already have a job and are looking for a new one. How do your observations change ? You can repeat this with different choices of categories to see how selecting different metrics can lead to different interpretations of the same data.


## Task 3:

Use the [amstat](https://www.amstat.ch) website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between *Swiss* and *foreign* workers.

The Economic Secretary (SECO) releases [a monthly report](https://www.seco.admin.ch/seco/fr/home/Arbeit/Arbeitslosenversicherung/arbeitslosenzahlen.html) on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for *foreign* (`5.1%`) and *Swiss* (`2.2%`) workers. 

Show the difference in unemployment rates between the two categories in each canton on a Choropleth map (*hint* The easy way is to show two separate maps, but can you think of something better ?). Where are the differences most visible ? Why do you think that is ?

Now let's refine the analysis by adding the differences between age groups. As you may have guessed it is nearly impossible to plot so many variables on a map. Make a bar plot, which is a better suited visualization tool for this type of multivariate data.

**Step 1:** Read the data from the CSV

In [92]:
df=pd.read_csv('../rate_Canton_Nationality')
df.dtypes
df=df.drop(52)
df.head()

,Index,region,Canton,Nationality,Unemploy_rate,Unemploy_rate_Young,Registered_unemployed,Young_Unemployed,Long_period_unemployed,Job_seekers,Job_seekers_Employed
0,0,Suisse alémanique,Zurich,Etrangers,5.3,5.8,12111,1015,2106,15384,3273
1,1,Suisse alémanique,Zurich,Suisses,2.5,3.2,15114,2405,2647,18772,3658
2,2,Suisse alémanique,Berne,Etrangers,5.5,7.7,4900,597,848,6859,1959
3,3,Suisse alémanique,Berne,Suisses,1.8,2.2,8758,1406,1488,11526,2768
4,4,Suisse alémanique,Lucerne,Etrangers,3.9,4.7,1593,217,181,2902,1309


In order to plot maps, we need some setup:

In [138]:
init_coords =[46.819, 8.223]
cantons_geo_path = r'ch-cantons.topojson.json'
geo_json_data = json.load(open(cantons_geo_path))

dic=dict(geo_json_data)
liste =dic['objects']['cantons']['geometries']
identification = []
for d in liste:
    identification.append(d['id'])

**Step 2:** We plot the choropleth for unemployment rates of Swiss nationals

In [139]:
df_swissmap = df[df.Nationality != "Etrangers"]
df_swissmap.head()

suisses_map = folium.Map(location=init_coords, tiles='cartodbpositron', zoom_start=7)



df_swissmap['id']=identification

swiss_dict = df_swissmap.set_index('id')['Unemploy_rate']


colormap = linear.OrRd.scale(0,df_swissmap.Unemploy_rate.max())

folium.TopoJson(open('ch-cantons.topojson.json'),
                'objects.cantons',
                style_function=lambda feature: {
                    'fillColor': colormap(swiss_dict[feature['id']]),
                    'color': 'black',
                    'weight': 1,
                    'dashArray': '5, 5',
                    'fillOpacity': 0.8,
                },name='Unemployement rates'
               ).add_to(suisses_map)

colormap.caption = 'Unemployement rates of Swiss people in Switzerland (%)'
colormap.add_to(suisses_map)
suisses_map



**Step 3:** We plot the choropleth for unemployment rates of foreign nationals

In [140]:
df_foreignermap = df[df.Nationality != "Suisses"]
df_foreignermap.head()

etranger_map = folium.Map(location=init_coords, tiles='cartodbpositron', zoom_start=7)

df_foreignermap['id']=identification

etranger_dict = df_foreignermap.set_index('id')['Unemploy_rate']


colormap_f = linear.OrRd.scale(0,df_foreignermap.Unemploy_rate.max())

folium.TopoJson(open('ch-cantons.topojson.json'),
                'objects.cantons',
                style_function=lambda feature: {
                    'fillColor': colormap_f(etranger_dict[feature['id']]),
                    'color': 'black',
                    'weight': 1,
                    'dashArray': '5, 5',
                    'fillOpacity': 0.8,
                },name='Unemployement rates'
               ).add_to(etranger_map)

colormap_f.caption = 'Unemployement rates of Foreigners in Switzerland (%)'
colormap_f.add_to(etranger_map)
etranger_map



**Step 4:** We plot the choropleth for showing the difference between Swiss and Foreign unemployment rates

In [170]:
df1 = df.copy()
df1['diffs'] = df1.groupby(['Canton'])['Unemploy_rate'].diff()
df1.sort_index(inplace=True)
df1.head(20)

df_diffmap = df1[np.isfinite(df1['diffs'])]
df_diffmap['unemployment_diff'] = 0
df_diffmap['unemployment_diff'][(df_diffmap['Nationality'] == 'Etrangers')] = -1*df_diffmap['diffs']
df_diffmap['unemployment_diff'][(df_diffmap['Nationality'] == 'Suisses')] = df_diffmap['diffs']
df_diffmap.head()

diff_map = folium.Map(location=init_coords, tiles='cartodbpositron', zoom_start=7)
df_diffmap['id']=identification
diff_dict = df_diffmap.set_index('id')['unemployment_diff']
x = max(abs(df_diffmap.unemployment_diff.min()),abs(df_diffmap.unemployment_diff.max()))

colormap_d = linear.RdBu.scale(-x,x)
folium.TopoJson(open('ch-cantons.topojson.json'),
                'objects.cantons',
                style_function=lambda feature: {
                    'fillColor': colormap_d(diff_dict[feature['id']]),
                    'color': 'black',
                    'weight': 1,
                    'dashArray': '5, 5',
                    'fillOpacity': 0.8,
                },name='Unemployement rates'
               ).add_to(diff_map)

colormap_d.caption = 'Difference in unemployment rates of Foreigners and Swiss people in Switzerland (%)'
colormap_d.add_to(diff_map)
diff_map

## Bonus:

Using the map you have just built, and the geographical information contained in it, could you give a *rough estimate* of the difference in unemployment rates between the areas divided by the [RÃ¶stigraben](https://en.wikipedia.org/wiki/R%C3%B6stigraben)?